# Import Libraries

In [1]:
import os
from os import listdir
from os.path import isfile, join
import xml.etree.cElementTree as ET
import re
import numpy as np
import pandas as pd

import nltk
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import csv
import argparse
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score
from sklearn import metrics, preprocessing


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

In [5]:
#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

In [11]:
#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
import datapane as dp 

import pickle 

In [14]:
pd.set_option('max_colwidth', 150)

In [18]:
tweets = pd.read_csv("largedata/tweets-6k.csv")
tweets.head(2)

,Tweets
0,Spoke at @AllHealthReform event on health care reform this morning. No live tweeting allowed. Did it really happen? @ddiamond True. You can cover ...
1,"Meaning achieved! Today, my desperate quest for meaning in life has brought me to Crystal Palace. #CRYSUN @SunderlandAFC https://t.co/ZaZaf2Lfig H..."


In [21]:
tweets = tweets["Tweets"].values.tolist()

In [23]:
tweets[0]

'Spoke at @AllHealthReform event on health care reform this morning. No live tweeting allowed. Did it really happen? @ddiamond True. You can cover \'more\' with \'more\' for \'less.\' But you don\'t want to be one of the \'less\' who get \'le… https://t.co/Ui54o2m5LD The hard part was not gawking at the Chrysler Building next door. My son built a 6 ft replica of it in our backyard. https://t.co/0aftXVUR3P @nicholas_bagley @onceuponA The other key question is which of these would hit Plan Year \'17 and which would hit Plan Year \'18. Must read from @nicholas_bagley and @onceuponA and must ask if the goal is orderly transition or disruption.\n\nhttps://t.co/PmAUOeMo6x @squelchedalot 11 generations all in VA in my family. Can\'t speak to all the branches though. #Obamacare Executive Order is not a specific action. It is a declaration of administrative war. Orderly transition or massive disruption? The President goes by. #InaugurationDay. https://t.co/RDQSx1VWq4 Motorcade in the parade. #I

In [24]:
tweets[1]

'Meaning achieved! Today, my desperate quest for meaning in life has brought me to Crystal Palace. #CRYSUN @SunderlandAFC https://t.co/ZaZaf2Lfig He\'ll be the judge of that. https://t.co/UerwkOSt1u It\'s bin day AND #transferdeadlineday. This won\'t happen again in our lifetime. #DrainTheSwamp https://t.co/NHSFjpkPMF @danthomascomedy Haha yes. It\'s not just impressive or anything. Genuinely enjoying it. Seems best not to question it as you\'re going along. @danthomascomedy Worth it.  Me and Chopping are in the 300s.  It\'s an astonishing book. Never read anything like it. @danthomascomedy That\'s a belter!  How\'s #DanReadsInfiniteJest going? @JasonArnopp @RetroGamer_Mag I thought she was going out with Wayne Of The Exploding Fist? @JasonArnopp @RetroGamer_Mag I loved Space Harriet.  How\'s she doing? That\'s why Jill Stein\'s in the new Adam Sandler movie. https://t.co/BR8FpK7xXb #Trump45 is the political equivalent of a "100% Unauthorised" boy band calendar I keep my Linkin Park Cr

In [26]:
len(tweets[0])

11441

# Tweets Preprocessing

In [27]:

replacement_patterns = [
    #match url (i.e: https://t.co/5tF5G9VKtq)
    (r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ''),

    #match user (i.e: @cerpintor )
    (r'@\w+', ''),

    #match hashtag (i.e: #WomensMarchOnWashington)
    (r'#\w+', ''),

    #Replace "&..." with ''
    (r'&\w+', '')
]


In [28]:
class RegexReplacer(object):
    def __init__(self, patterns = replacement_patterns):
        self.patterns = [(re.compile(regrex), repl) for (regrex, repl) in patterns]
        
    # Replace the words that match the patterns with replacement words
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
            return s

In [38]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in emoji.distinct_emoji_list(text)]
    #emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [30]:
def lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    
    doc = nlp(" ".join(text)) 
    return [token.lemma_ for token in doc if token.pos_ in allowed_postags]

In [34]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
tknz = TweetTokenizer()
replacer = RegexReplacer()
# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']
# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)
# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

punc = string.punctuation

In [35]:

# Build the bigram and trigram models
bigram = gensim.models.Phrases(tweets, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tweets], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [36]:

def normalize(doc):
    
    for i in range(len(doc)):
        
        doc[i] = give_emoji_free_text(doc[i])
        
        #Tokenize with replacement
        doc[i] = tknz.tokenize(replacer.replace(doc[i]))
        
        #Filter stopwords, punctuations, and lowercase
        doc[i] = [w.lower() for w in doc[i] if w not in punc and w not in ALL_STOP_WORDS]
    
        # Bigram
        doc[i] = bigram_mod[doc[i]]
        
        #lemmatizer = WordNetLemmatizer()
        
        doc[i] = lemmatization(doc[i], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

        
        
        #concat
#         doc[i] = ' '.join(w for w in doc[i])
        
    return doc

In [ ]:
 def is_emoji(string): 
     """Returns True if the string is an emoji.""" 
     return string in unicode_codes.EMOJI_DATA 

In [39]:
tweets = normalize(tweets)

In [40]:

tweets[0] 

['speak',
 'health',
 'care',
 'reform',
 'morning',
 'live',
 'tweeting',
 'allow',
 'happen',
 'true',
 'cover',
 'hard',
 'gawk',
 'door',
 'son',
 'build',
 'backyard',
 '@nicholas_bagley',
 'key',
 'question',
 'hit',
 'plan',
 'year',
 'hit',
 'plan',
 'year',
 'must',
 'read',
 'ask',
 'orderly',
 'transition',
 'disruption',
 'generation',
 'family',
 'can',
 'speak',
 'branch',
 'executive',
 'order',
 'specific',
 'action',
 'declaration',
 'administrative',
 'orderly',
 'transition',
 'massive',
 'disruption',
 'president',
 'go',
 'motorcade',
 'inaugurationday',
 'protester',
 'chant',
 'really',
 'motorcade',
 'head',
 '#',
 'inaugurationday',
 'protester',
 'many',
 'slice',
 '#',
 'inaugurationday',
 'read',
 'work',
 'solution',
 'trade',
 'off',
 'view',
 'chant',
 'singe',
 'scene',
 '#',
 'inaugurationday',
 'start',
 'spread',
 'news',
 'bring',
 'act',
 '#',
 'fda2017',
 'quick',
 'thought',
 'ask',
 'how',
 'view',
 'grind',
 'poster',
 'reference',
 'interview',